In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


#preprocesing
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Removing warnings



import warnings
warnings.filterwarnings('ignore')

# Storing plots

%matplotlib inline

In [ ]:
# pip install catboost


In [ ]:
train_dataset = pd.read_feather('/kaggle/input/amexfeather/train_data.ftr')

In [ ]:
train_dataset = train_dataset.set_index('customer_ID', drop=True)

In [ ]:
all_columns = [col for col in train_dataset.columns]

In [ ]:
# train_dataset = train_dataset.groupby('customer_ID').tail(1).set_index('customer_ID', drop=True).sort_index()

In [ ]:
# train_dataset

In [ ]:
# train_dataset.describe()

In [ ]:
train_dataset.shape

In [ ]:
train_dataset.isnull().sum()


In [ ]:
min_null_count =  int(((100-25)/100)*train_dataset.shape[0] + 1)
train_dataset = train_dataset.dropna( axis=1, 
                thresh=min_null_count)

In [ ]:
after_null_columns = [col for col in train_dataset.columns]

In [ ]:
removed= [ col for col in all_columns if  col not in after_null_columns ]

In [ ]:
removed

In [ ]:
train_dataset.shape

In [ ]:
# for column in train_dataset.columns:
#     print(column,"----->", train_dataset[column].unique())

In [ ]:
train_dataset.drop(["S_2"],axis=1,inplace=True)

In [ ]:
# train_dataset.drop(["customer_ID"],axis=1,inplace=True)

In [ ]:
categories=[]
for categorical_column in train_dataset.select_dtypes(include=['category','object']).columns:
    categories.append(categorical_column)
categories

In [ ]:
# enc = LabelEncoder()
# for categorical_column in train_dataset.select_dtypes(include=['category','object']).columns:
#     print(train_dataset[categorical_column].head())
#     features[categorical_column]=enc.fit_transform(features[categorical_column]

In [ ]:
enc = LabelEncoder()
for categorical_column in train_dataset.select_dtypes(include=['category','object']).columns:
    train_dataset[categorical_column]=enc.fit_transform(train_dataset[categorical_column])

In [ ]:
# train_dataset["D_63"]=enc.fit_transform(train_dataset["D_63"])

In [ ]:
# train_dataset["D_64"]=enc.fit_transform(train_dataset["D_64"])

In [ ]:
# train_dataset["D_64"]=train_dataset["D_64"].fillna(train_dataset["D_64"].mode())

In [ ]:
# train_dataset["D_68"].isnull().sum()

In [ ]:
for category in categories:
    train_dataset[category]=train_dataset[category].fillna(train_dataset[category].mode())

In [ ]:
train_dataset["P_2"].isnull().sum()

In [ ]:
train_dataset["P_2"].mean()

In [ ]:
for column in train_dataset.columns:
    if(column not in categories):
#         print(column)
        train_dataset[column]=train_dataset[column].fillna(train_dataset[column].median())

In [ ]:
X = train_dataset.iloc[:, :-1]
y = train_dataset.iloc[:, -1:]

In [ ]:
y.head()

In [ ]:
del train_dataset


In [ ]:
import gc

gc.collect()

In [ ]:
X1 = X.groupby('customer_ID').mean()

In [ ]:
X2 = X.groupby('customer_ID').max()

In [ ]:
X3 = X.groupby('customer_ID').min()

In [ ]:
# X4 = X.groupby('customer_ID').agg(pd.Series.mode)

In [ ]:
X5=X.groupby('customer_ID').tail(1)

In [ ]:
X1. columns += '_mean' 

In [ ]:
X2. columns += '_max' 

In [ ]:
X3. columns += '_min' 

In [ ]:
# X4. columns += '_mode' 

In [ ]:
X5. columns += '_latest' 

In [ ]:
X6=pd.merge(X1, X2, on='customer_ID')

In [ ]:
X6=pd.merge(X6, X3, on='customer_ID')

In [ ]:
X6=pd.merge(X6, X5, on='customer_ID')

In [ ]:
X.head()

In [ ]:
cor_matrix = X6.corr()
col_core = set()

for i in range(len(cor_matrix.columns)):
    for j in range(i):
        if(cor_matrix.iloc[i, j] > 0.9):
            col_name = cor_matrix.columns[i]
            col_core.add(col_name)
col_core

In [ ]:
X6 = X6.drop(col_core, axis=1)

In [ ]:
y= y.groupby('customer_ID').mean()

In [ ]:
y.head()

In [ ]:
y=y.round(0).astype(int)

In [ ]:
categories

In [ ]:
for category in categories:
    X6[category+"_mean"]=X6[category+"_mean"].round(0).astype(int)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X6,y,test_size=0.3, random_state=23)

In [ ]:
x_train.isnull().sum()

# LogisticRegression

In [ ]:
# model = LogisticRegression(max_iter=1000)
# model.fit(x_train, y_train)

# LinearRegression

In [ ]:
# model = LinearRegression()
# model.fit(x_train, y_train)

# lightgbm

In [ ]:
categorical_feature=[]
for category in categories:
    for i in ["mean", "max", "min", "latest"]:
        col=category+"_"+i
        if col not in col_core:
            categorical_feature.append(col)

In [ ]:
import lightgbm as lgb

d_train = lgb.Dataset(x_train, label=y_train, categorical_feature = categorical_feature)

params = {'objective': 'binary','n_estimators': 1200,'metric': 'binary_logloss','boosting': 'gbdt','num_leaves': 90,'reg_lambda' : 50,'colsample_bytree': 0.19,'learning_rate': 0.03,'min_child_samples': 2400,'max_bins': 511,'seed': 42,'verbose': -1}

# trained model with 100 iterations
model = lgb.train(params, d_train, 100)

# CatBoostRegressor

In [ ]:
# from catboost import CatBoostRegressor
# #build model
# model = CatBoostRegressor(cat_features = categories)
# # Fit model 
# model.fit( x_train, y_train,
#                eval_set=(x_test, y_test),
#                plot=True)

# xgboost

In [ ]:
# from xgboost import XGBRegressor
# model = XGBRegressor()
# model.fit(x_train, y_train)

In [ ]:
# model = XGBRegressor(n_estimators=500)
# model.fit(x_train, y_train, 
#              early_stopping_rounds=5, 
#              eval_set=[(x_test, y_test)],
#              verbose=False)

In [ ]:
# xgb1 = XGBRegressor()
# parameters = {'nthread':[4], 
#               'objective':['reg:linear'],
#               'learning_rate': [.4, .03, .02,.04,.05], 
#               'max_depth': [3,4,6, 7],
#               'min_child_weight': [4],
#               'silent': [1],
#               'subsample': [0.7],
#               'colsample_bytree': [0.7],
#               'n_estimators': [1000]}

# parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
#               'objective':['reg:linear'],
#               'learning_rate': [.04], #so called `eta` value
#               'max_depth': [8],
#               'min_child_weight': [4],
#               'silent': [1],
#               'subsample': [0.7],
#               'colsample_bytree': [0.7],
#               'n_estimators': [1250]}

# parameters={'colsample_bytree': [0.7], 'learning_rate': [0.04], 'max_depth':[8],
# 'min_child_weight':[4], 'n_estimators': [1250], 'nthread': [4], 'objective':
#             ['reg:linear'], 'silent': [1], 'subsample':[ 0.7]}

# parameters={"learning_rate" :[0.1],
#  "n_estimators":[1000],
#  "max_depth":[5],
#  "min_child_weight":[1],
#  "gamma":[0],
#  "subsample":[0.8],
#  "colsample_bytree":[0.8],
#  "objective": ['reg:linear'],
#  "nthread":[4],
#  "scale_pos_weight":[1],
#  "seed":[27]
# }

# xgb_grid = GridSearchCV(xgb1,
#                         parameters,
#                         cv = 2,
#                         n_jobs = 5,
#                         verbose=True)

# xgb_grid.fit(x_train,
#          y_train)

# print(xgb_grid.best_score_)
# print(xgb_grid.best_params_)

In [ ]:
test_dataset = pd.read_feather('/kaggle/input/amexfeather/test_data.ftr')

In [ ]:
test_dataset = test_dataset.set_index('customer_ID', drop=True)

In [ ]:
test_dataset = test_dataset.drop(removed, axis=1)

In [ ]:
test_dataset.drop(["S_2"],axis=1,inplace=True)

In [ ]:
enc = LabelEncoder()
for categorical_column in test_dataset.select_dtypes(include=['category','object']).columns:
    test_dataset[categorical_column]=enc.fit_transform(test_dataset[categorical_column])

In [ ]:
for category in categories:
    test_dataset[category]=test_dataset[category].fillna(test_dataset[category].mode())

In [ ]:
for column in test_dataset.columns:
    if(column not in categories):
#         print(column)
        test_dataset[column]=test_dataset[column].fillna(test_dataset[column].median())

In [ ]:
test_dataset1 = test_dataset.groupby('customer_ID').mean()

In [ ]:
test_dataset2 = test_dataset.groupby('customer_ID').max()

In [ ]:
test_dataset3 = test_dataset.groupby('customer_ID').min()

In [ ]:
test_dataset5=test_dataset.groupby('customer_ID').tail(1)

In [ ]:
test_dataset1. columns = test_dataset.columns+'_mean' 
test_dataset2. columns = test_dataset.columns+'_max' 
test_dataset3. columns = test_dataset.columns+'_min' 
test_dataset5. columns = test_dataset.columns+'_latest'

In [ ]:
test_dataset6=pd.merge(test_dataset1, test_dataset2, on='customer_ID')

In [ ]:
test_dataset6=pd.merge(test_dataset6, test_dataset3, on='customer_ID')

In [ ]:
test_dataset6=pd.merge(test_dataset6, test_dataset5, on='customer_ID')

In [ ]:
test_dataset6 = test_dataset6.drop(col_core, axis=1)

In [ ]:
for category in categories:
    test_dataset6[category+"_mean"]=test_dataset6[category+"_mean"].round(0).astype(int)

In [ ]:
y_pred = model.predict(test_dataset6)

In [ ]:
# def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

#     def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
#         df = (pd.concat([y_true, y_pred], axis='columns')
#               .sort_values('prediction', ascending=False))
#         df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
#         four_pct_cutoff = int(0.04 * df['weight'].sum())
#         df['weight_cumsum'] = df['weight'].cumsum()
#         df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
#         return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
#     def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
#         df = (pd.concat([y_true, y_pred], axis='columns')
#               .sort_values('prediction', ascending=False))
#         df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
#         df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
#         total_pos = (df['target'] * df['weight']).sum()
#         df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
#         df['lorentz'] = df['cum_pos_found'] / total_pos
#         df['gini'] = (df['lorentz'] - df['random']) * df['weight']
#         return df['gini'].sum()

#     def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
#         y_true_pred = y_true.rename(columns={'target': 'prediction'})
#         return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

#     g = normalized_weighted_gini(y_true, y_pred)
#     d = top_four_percent_captured(y_true, y_pred)

#     return 0.5 * (g + d)

In [ ]:
# print(amex_metric(pd.DataFrame({'target':y_test["target"].values}), pd.DataFrame({'prediction':y_pred})))

In [ ]:
def amex_metric(y_true, y_pred):
    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

In [ ]:
print(amex_metric(y_test["target"].values,y_pred))

In [ ]:
output = pd.DataFrame({'customer_ID': test_dataset6.index, 'prediction': y_pred})


In [ ]:
cd /kaggle/working/

In [ ]:
output.to_csv('submission11.csv', index=False)